In [1]:
import matplotlib.pyplot as plt
import os, warnings
import pandas as pd
import numpy as np

import keras
from keras import layers
import tensorflow as tf
from custom_functions import custom_image_dataset_for_pdd_model


def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    label = tf.cast(label, dtype=tf.float32)
    return image, label


def set_seed(seed=19893):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_DETERMINISTIC_OPS"] = "1"


set_seed()
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlesize=16, titlepad=10)
plt.rc("image", cmap="magma")
warnings.filterwarnings("ignore")
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:

pre_trained_model = 'C:/Users/USER/Documents/models/cv-course-models/cv-course-models/vgg16-pretrained-base'
train_data_path = "C:/Users/USER/Documents/datasets/plant disease recognition dataset/Train/Train/plants.csv"
train_image_path = "C:/Users/USER/Documents/datasets/plant disease recognition dataset/Train/Train"
test_data_path = "C:/Users/USER/Documents/datasets/plant disease recognition dataset/Train/Train/plants.csv"
test_image_path = "C:/Users/USER/Documents/datasets/plant disease recognition dataset/Train/Train"

In [3]:
train_, test_, label_mapping_df = custom_image_dataset_for_pdd_model(train_data_path, train_image_path, test_data_path, test_image_path)

train_ = (train_.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))
test_ = (test_.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))

Beginning Preprocessing Images...
Length of unique_values: 3
Length of encoded_labels_train: 1322
Length of encoded_labels_test: 1322
Done Preprocessing Images...


In [11]:
pretrained_base = keras.models.load_model(pre_trained_model)
pretrained_base.trainable = False

In [18]:
model = keras.Sequential([
    pretrained_base,
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=[128, 128, 3]),
    layers.MaxPool2D(),
    
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    layers.Flatten(),
    layers.Dense(units=64, activation="relu"),
    layers.Dense(units=len(label_mapping_df), activation="softmax"),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])


In [19]:
history = model.fit(train_, validation_data=test_, epochs=10, verbose=2)
print("Done training of model...")

Epoch 1/10


ValueError: in user code:

    File "c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Users\USER\AppDat

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

save_dir = "C:/Users/USER/Documents/models"
os.makedirs(save_dir, exist_ok=True)
model.save(os.path.join(save_dir, "plant_disease_detection_model.h5"))
plt.show()